In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
        # print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install datasets peft torch torchvision 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

In [3]:
!pip install -q transformers torch evaluate scikit-learn pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.3 MB/s eta 0:00:00


In [4]:
import os
import ast
import pandas as pd
from PIL import Image
import torch
from sklearn.model_selection import train_test_split  # scikit-learn split :contentReference[oaicite:2]{index=2}
import evaluate                                        # metrics :contentReference[oaicite:3]{index=3}
from transformers import Blip2Processor, Blip2ForConditionalGeneration , BitsAndBytesConfig, get_linear_schedule_with_warmup
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel

CSV_PATH = "/kaggle/input/images-with-vqas/merged_image_data_vqa.csv"
df = pd.read_csv(CSV_PATH, nrows = 100)

df["image_path"] = df["image_path"].str.replace(
    r"^Dataset/final_dataset/",
    "/kaggle/input/images-with-vqas/final_dataset/final_dataset/",
    regex=True
)

df["vqa_response"] = df["vqa_response"].apply(ast.literal_eval)
df = df.explode("vqa_response").reset_index(drop=True)
df[["question", "answer"]] = pd.DataFrame(df["vqa_response"].tolist(), index=df.index)

exists = df["image_path"].apply(os.path.exists)
print(f"Skipping {(~exists).sum()} rows with missing images")
df = df[exists]

df = df[["image_path", "question", "answer"]].reset_index(drop=True)
print(f"Total examples: {len(df)}")


2025-05-12 20:17:24.858163: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747081045.075559      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747081045.136447      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Skipping 0 rows with missing images
Total examples: 299


In [5]:
df.head(5)

,image_path,question,answer
0,/kaggle/input/images-with-vqas/final_dataset/f...,What is the main color of the shoe?,Brown
1,/kaggle/input/images-with-vqas/final_dataset/f...,What is the pattern on the shoe?,Snakeskin
2,/kaggle/input/images-with-vqas/final_dataset/f...,Does the shoe have a tassel?,Yes
3,/kaggle/input/images-with-vqas/final_dataset/f...,What color are the drawer slides?,White
4,/kaggle/input/images-with-vqas/final_dataset/f...,What appears to be the material of the slides?,Metal


In [6]:
# from transformers import Blip2Processor, Blip2ForConditionalGeneration
# import torch
# from PIL import Image

processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
model     = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-flan-t5-xl",
    torch_dtype=torch.float16,
    device_map="auto"
)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.eval().to(device)

# import requests
# from PIL import Image
# from transformers import BlipProcessor, BlipForQuestionAnswering

# processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-capfilt-large")
# model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-capfilt-large")
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
)

# model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval().to(device)




Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/5.81G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

trainable params: 473,088 || all params: 3,942,919,680 || trainable%: 0.0120


PeftModel(
  (base_model): LoraModel(
    (model): Blip2ForConditionalGeneration(
      (vision_model): Blip2VisionModel(
        (embeddings): Blip2VisionEmbeddings(
          (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
        )
        (encoder): Blip2Encoder(
          (layers): ModuleList(
            (0-38): 39 x Blip2EncoderLayer(
              (self_attn): Blip2Attention(
                (dropout): Dropout(p=0.0, inplace=False)
                (qkv): Linear(in_features=1408, out_features=4224, bias=True)
                (projection): Linear(in_features=1408, out_features=1408, bias=True)
              )
              (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
              (mlp): Blip2MLP(
                (activation_fn): GELUActivation()
                (fc1): Linear(in_features=1408, out_features=6144, bias=True)
                (fc2): Linear(in_features=6144, out_features=1408, bias=True)
              )
              

In [7]:
from torch.utils.data import DataLoader
class VQADataset(torch.utils.data.Dataset):
    def __init__(self, df, processor, max_length=32):
        self.df = df
        self.processor = processor
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row.image_path).convert("RGB")
        question, answer = row.question, row.answer

        # BLIP-VQA expects pixel_values + input_ids + attention_mask
        enc = self.processor(
            image, question,
            padding="max_length", truncation=True,
            max_length=self.max_length, return_tensors="pt"
        )
        labels = self.processor.tokenizer(
            answer,
            padding="max_length", truncation=True,
            max_length=self.max_length, return_tensors="pt"
        ).input_ids
        labels[labels == self.processor.tokenizer.pad_token_id] = -100

        item = {
            "pixel_values": enc.pixel_values.squeeze(),
            "input_ids":    enc.input_ids.squeeze(),
            "attention_mask": enc.attention_mask.squeeze(),
            "labels":       labels.squeeze()
        }
        return item

train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

train_ds = VQADataset(train_df, processor)
val_ds   = VQADataset(val_df, processor)

train_dataloader = DataLoader(train_ds, batch_size=12, shuffle=True,  pin_memory=True)
valid_dataloader   = DataLoader(val_ds,   batch_size=12, shuffle=False, pin_memory=True)


In [8]:
import torch
import os
from tqdm import tqdm
import pickle
# from transformers import Blip2Processor, Blip2ForConditionalGeneration # Assuming loaded
# Assuming base_model, lora_config, get_peft_model are setup
# Assuming train_dataloader, valid_dataloader are ready

# --- Your existing setup ---
# Ensure model is the PEFT model: model = get_peft_model(base_model, lora_config)
model = model.to(device) # Ensure model is on device

optimizer = torch.optim.AdamW(model.parameters(), lr=4e-5) # Keep initial LR for now, or lower it (e.g., 1e-5)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9, last_epoch=-1, verbose=False)
num_epochs = 1 # Or more
patience = 10
min_eval_loss = float("inf")
early_stopping_hook = 0
tracking_information = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Use the recommended way for GradScaler
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available()) # Enable only if cuda is available

# Define a max gradient norm value
max_grad_norm = 1.0 # A common value, adjust if needed

output_dir = "/kaggle/working/blip2_lora"
os.makedirs(output_dir, exist_ok=True)

# --- Training Loop ---
for epoch in range(num_epochs):
    epoch_loss = 0
    model.train()
    for idx, batch in zip(tqdm(range(len(train_dataloader)), desc=f'Epoch {epoch+1} Training'), train_dataloader):
        # Move data to device
        try:
            input_ids = batch.pop('input_ids').to(device)
            pixel_values = batch.pop('pixel_values').to(device)
            attention_mask = batch.pop('attention_mask').to(device)
            labels = batch.pop('labels').to(device)
        except KeyError as e:
            print(f"Missing key in batch: {e}")
            continue # Skip batch if keys are missing
        except Exception as e:
            print(f"Error loading batch data: {e}")
            continue # Skip problematic batch


        optimizer.zero_grad()

        with torch.amp.autocast(device_type=device.type, dtype=torch.float16, enabled=torch.cuda.is_available()):
            outputs = model(input_ids=input_ids,
                            pixel_values=pixel_values,
                            attention_mask=attention_mask, # Include mask if model expects it during training
                            labels=labels)
            loss = outputs.loss

        if loss is None:
             print(f"Warning: Loss is None for batch {idx}. Skipping backward pass.")
             continue

        if torch.isnan(loss) or torch.isinf(loss):
            print(f"Warning: Detected NaN/Inf loss at batch {idx} before scaling. Skipping batch.")
            # Optionally: investigate the batch content here
            # print("Batch keys:", list(batch.keys())) # See if anything unexpected remained
            # print("Input IDs shape:", input_ids.shape)
            # print("Labels shape:", labels.shape)
            continue # Skip the backward pass for this batch

        # Scale loss and calculate gradients
        scaler.scale(loss).backward()

        # ** ADD GRADIENT CLIPPING HERE **
        # Unscale gradients before clipping
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        # Optimizer step
        scaler.step(optimizer)

        # Update scaler
        scaler.update()

        # Log loss (check if item() causes issues with NaN/Inf, though check above should prevent)
        if not (torch.isnan(loss) or torch.isinf(loss)):
             epoch_loss += loss.item()
        else:
             print(f"NaN/Inf loss recorded at step {idx}, not adding to epoch_loss")


    # --- Validation Loop (mostly unchanged) ---
    model.eval()
    eval_loss = 0
    with torch.no_grad(): # Ensure no gradients during validation
        for idx, batch in zip(tqdm(range(len(valid_dataloader)), desc=f'Epoch {epoch+1} Validating'), valid_dataloader):
            try:
                input_ids = batch.pop('input_ids').to(device)
                pixel_values = batch.pop('pixel_values').to(device)
                attention_mask = batch.pop('attention_mask').to(device)
                labels = batch.pop('labels').to(device)
            except KeyError as e:
                print(f"Missing key in validation batch: {e}")
                continue
            except Exception as e:
                print(f"Error loading validation batch data: {e}")
                continue


            with torch.amp.autocast(device_type=device.type, dtype=torch.float16, enabled=torch.cuda.is_available()):
                outputs = model(input_ids=input_ids,
                            pixel_values=pixel_values,
                            attention_mask=attention_mask,
                            labels=labels)
                loss = outputs.loss

            if loss is not None and not (torch.isnan(loss) or torch.isinf(loss)):
                eval_loss += loss.item()
            else:
                 print(f"NaN/Inf/None loss detected during validation step {idx}")


    # Calculate average losses
    # Handle potential division by zero if dataloaders are empty
    avg_train_loss = epoch_loss / len(train_dataloader) if len(train_dataloader) > 0 else float('nan')
    avg_eval_loss = eval_loss / len(valid_dataloader) if len(valid_dataloader) > 0 else float('nan')
    current_lr = optimizer.param_groups[0]["lr"]

    print(f"Epoch: {epoch+1} - Training loss: {avg_train_loss:.4f} - Eval Loss: {avg_eval_loss:.4f} - LR: {current_lr}")

    # Check for NaN/Inf in avg_eval_loss before comparison
    if not (torch.isnan(torch.tensor(avg_eval_loss)) or torch.isinf(torch.tensor(avg_eval_loss))): # Convert to tensor for robust check
        tracking_information.append((avg_train_loss, avg_eval_loss, current_lr))
        scheduler.step() # Step scheduler only on valid epochs

        # Saving logic - Now should trigger if loss is valid and decreasing
        if avg_eval_loss < min_eval_loss:
            best_model_dir = os.path.join(output_dir, "best-checkpoint")
            # save_pretrained on a PeftModel saves the adapter config and weights
            model.save_pretrained(best_model_dir)
            print(f"Saved PEFT adapter checkpoint to {best_model_dir}")
            min_eval_loss = avg_eval_loss
            early_stopping_hook = 0
        else:
            early_stopping_hook += 1
            print(f"Validation loss did not improve. Early stopping counter: {early_stopping_hook}/{patience}")
            if early_stopping_hook >= patience:
                print(f"Early stopping triggered after {patience} epochs without improvement.")
                break
    else:
        print(f"Skipping saving and scheduler step due to NaN/Inf validation loss in Epoch {epoch+1}.")
        # Optionally break or implement other logic if NaNs persist
        # break

pickle.dump(tracking_information, open("tracking_information.pkl", "wb"))
print("The finetuning process has completed.")

/tmp/ipykernel_19/1745122541.py:22: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available()) # Enable only if cuda is available
Epoch 1 Training:   4%|▍         | 1/23 [00:02<01:01,  2.80s/it]

Epoch 1 Training:   9%|▊         | 2/23 [00:04<00:43,  2.09s/it]

Epoch 1 Training:  13%|█▎        | 3/23 [00:05<00:37,  1.86s/it]

Epoch 1 Training:  17%|█▋        | 4/23 [00:07<00:33,  1.76s/it]

Epoch 1 Training:  22%|██▏       | 5/23 [00:09<00:30,  1.71s/it]

...

Epoch 1 Training:  91%|█████████▏| 21/23 [00:34<00:03,  1.56s/it]

Epoch 1 Training:  96%|█████████▌| 22/23 [00:35<00:01,  1.56s/it]

Epoch 1 Training: 100%|██████████| 23/23 [00:36<00:00,  1.59s/it]


Epoch 1 Validating: 100%|██████████| 3/3 [00:03<00:00,  1.31s/it]

Epoch: 1 - Training loss: 0.0000 - Eval Loss: 4.6066 - LR: 4e-05
Saved PEFT adapter checkpoint to /kaggle/working/blip2_lora/best-checkpoint
The finetuning process has completed.


In [9]:
# import pickle
# import torch
# from PIL import Image
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# from transformers import Blip2Processor, Blip2ForConditionalGeneration
# from peft import PeftModel

# # 1) Load your test DataFrame `df_test` ... same preprocessing/explode as before
# #    e.g. df_test = load_data(csv_path).sample(500) or your dedicated test split

# # 2) Reload processor, base model, and LoRA adapter
# processor    = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
# base_model   = Blip2ForConditionalGeneration.from_pretrained(
#                    "Salesforce/blip2-flan-t5-xl", 
#                    torch_dtype=torch.float16
#                )
# peft_model   = PeftModel.from_pretrained(
#                    base_model, 
#                    os.path.join(output_dir, "best-checkpoint")  # same folder you saved earlier
#                )
# device       = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model        = peft_model.to(device).eval()

# preds, refs = [], []
# for row in val_df.itertuples():
#     img = Image.open(row.image_path).convert("RGB")
#     prompt = f"Based on the image, answer the following question with a single word. Question: {row.question} Answer:"
#     inputs = processor(images=img, text=prompt, return_tensors="pt").to(device)
#     out = model.generate(**inputs, max_new_tokens=10)
#     text = processor.batch_decode(out, skip_special_tokens=True)[0].strip()
#     pred = text.split()[0].rstrip(".,;:!?") if text else ""
#     preds.append(pred.lower())
#     refs.append(row.answer.lower())

import pickle
import torch
import os # Make sure os is imported
from PIL import Image
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support # Keep if you calculate metrics later

from transformers import Blip2Processor, Blip2ForConditionalGeneration
from peft import PeftModel

# Define output_dir if not already defined in this cell/scope
output_dir = "/kaggle/working/blip2_lora"
# Define val_df (your validation dataframe) if not already defined
# Example: val_df = ... load your data ...

# 1) Reload processor and BASE model
# processor    = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
# base_model   = Blip2ForConditionalGeneration.from_pretrained(
#                    "Salesforce/blip2-flan-t5-xl",
#                    # load_in_8bit=True, # Optional: if you trained with 8bit
#                    torch_dtype=torch.float16 # Ensure consistency if using float16
#                )

# 2) Load the LoRA adapter weights ON TOP of the base model
adapter_path = os.path.join(output_dir, "best-checkpoint")
print(f"Loading PEFT adapter from: {adapter_path}")
model = PeftModel.from_pretrained(model, adapter_path) # This should now work

# 3) Prepare model for inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval() # Set to evaluation mode

print("Model loaded successfully!")

# --- Your inference loop ---
preds, refs = [], []
# Make sure val_df exists and has 'image_path', 'question', 'answer' columns
for row in tqdm(val_df.itertuples(), total=len(val_df), desc="Generating Predictions"):
    print("testing")
    try:
        img = Image.open(row.image_path).convert("RGB")
        prompt = f"Based on the image, answer the following question with a single word. Question: {row.question} Answer:"
        # Ensure processor is sending tensors to the correct device and dtype if needed
        inputs = processor(images=img, text=prompt, return_tensors="pt").to(device, torch.float16) # Match model dtype

        # Generate output
        with torch.no_grad():
            out = model.generate(**inputs, max_new_tokens=10, do_sample=False) # Use do_sample=False for deterministic output

        # Decode and process prediction
        text = processor.batch_decode(out, skip_special_tokens=True)[0].strip()
        # Improved parsing: handle potential empty strings or multiple words robustly
        pred_words = text.split()
        pred = pred_words[0].rstrip(".,;:!?") if pred_words else "" # Get first word if exists

        preds.append(pred.lower())
        refs.append(str(row.answer).lower()) # Ensure answer is string and lower case
    except FileNotFoundError:
        print(f"Warning: Image not found at {row.image_path}. Skipping.")
        preds.append("[ImageNotFound]") # Add placeholder
        refs.append(str(row.answer).lower())
    except Exception as e:
        print(f"Error processing row {row.Index}: {e}")
        preds.append("[Error]") # Add placeholder
        refs.append(str(row.answer).lower())



Loading PEFT adapter from: /kaggle/working/blip2_lora/best-checkpoint


/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.base_model.model.qformer.encoder.layer.0.attention.attention.query.lora_A.default.weight', 'base_model.model.base_model.model.qformer.encoder.layer.0.attention.attention.query.lora_B.default.weight', 'base_model.model.base_model.model.qformer.encoder.layer.0.attention.attention.value.lora_A.default.weight', 'base_model.model.base_model.model.qformer.encoder.layer.0.attention.attention.value.lora_B.default.weight', 'base_model.model.base_model.model.qformer.encoder.layer.0.crossattention.attention.query.lora_A.default.weight', 'base_model.model.base_model.model.qformer.encoder.layer.0.crossattention.attention.query.lora_B.default.weight', 'base_model.model.base_model.model.qformer.encoder.layer.0.crossattention.attention.value.lora_A.default.weight', 'base_model.model.base_model.model.qformer.encoder.layer.0.crossattention.attention.val

Model loaded successfully!


Generating Predictions:   0%|          | 0/30 [00:00<?, ?it/s]

testing


...

Generating Predictions:  93%|█████████▎| 28/30 [00:07<00:00,  3.44it/s]

testing


Generating Predictions:  97%|█████████▋| 29/30 [00:08<00:00,  3.47it/s]

testing


Generating Predictions: 100%|██████████| 30/30 [00:08<00:00,  3.59it/s]


In [10]:
# ──────────────────────────────────────────────────────────────────────────────
# 6. Compute Metrics (and identify mismatches)
# ──────────────────────────────────────────────────────────────────────────────
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import evaluate
# from bert_score import BERTScorer # Keep if you add BERTScore later
# from rouge_score import rouge_scorer # Keep if you add ROUGE later
# from rapidfuzz.distance import Levenshtein # Keep if you add Levenshtein later
from sentence_transformers import SentenceTransformer, util # Keep if you add SBERT later
import numpy as np
import pandas as pd # Ensure pandas is imported if df is used



# normalize case
preds_l = [str(p).lower().strip() for p in preds] # Added str() and strip() for robustness
refs_l  = [str(r).lower().strip() for r in refs]   # Added str() and strip()

# 1) TOKEN-LEVEL EXACT MATCH → binary labels
y_pred_bin = [int(p == r) for p, r in zip(preds_l, refs_l)]
y_true_bin = [1]*len(refs_l)            # reference is always “correct” class

acc = accuracy_score(y_true_bin, y_pred_bin)
prec, rec, f1, _ = precision_recall_fscore_support(
    y_true_bin, y_pred_bin, average="binary", zero_division=0
)
print(f"Exact-match Accuracy: {acc:.3f}")
print(f"Exact-match Precision: {prec:.3f}") # Will be 1.0 if acc > 0, else 0.0
print(f"Exact-match Recall:    {rec:.3f}")    # Will be same as acc
print(f"Exact-match F1:        {f1:.3f}\n")  

Exact-match Accuracy: 0.367
Exact-match Precision: 1.000
Exact-match Recall:    0.367
Exact-match F1:        0.537



In [11]:
preds, refs = [], []
for row in train_df.itertuples():
    img = Image.open(row.image_path).convert("RGB")
    prompt = f"Based on the image, answer the following question with a single word. Question: {row.question} Answer:"
    inputs = processor(images=img, text=prompt, return_tensors="pt").to(device)
    out = model.generate(**inputs, max_new_tokens=10)
    text = processor.batch_decode(out, skip_special_tokens=True)[0].strip()
    pred = text.split()[0].rstrip(".,;:!?") if text else ""
    preds.append(pred.lower())
    refs.append(row.answer.lower())

In [12]:
# ──────────────────────────────────────────────────────────────────────────────
# 6. Compute Metrics (and identify mismatches)
# ──────────────────────────────────────────────────────────────────────────────
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import evaluate
# from bert_score import BERTScorer # Keep if you add BERTScore later
# from rouge_score import rouge_scorer # Keep if you add ROUGE later
# from rapidfuzz.distance import Levenshtein # Keep if you add Levenshtein later
from sentence_transformers import SentenceTransformer, util # Keep if you add SBERT later
import numpy as np
import pandas as pd # Ensure pandas is imported if df is used



# normalize case
preds_l = [str(p).lower().strip() for p in preds] # Added str() and strip() for robustness
refs_l  = [str(r).lower().strip() for r in refs]   # Added str() and strip()

# 1) TOKEN-LEVEL EXACT MATCH → binary labels
y_pred_bin = [int(p == r) for p, r in zip(preds_l, refs_l)]
y_true_bin = [1]*len(refs_l)            # reference is always “correct” class

acc = accuracy_score(y_true_bin, y_pred_bin)
prec, rec, f1, _ = precision_recall_fscore_support(
    y_true_bin, y_pred_bin, average="binary", zero_division=0
)
print(f"Exact-match Accuracy: {acc:.3f}")
print(f"Exact-match Precision: {prec:.3f}") # Will be 1.0 if acc > 0, else 0.0
print(f"Exact-match Recall:    {rec:.3f}")    # Will be same as acc
print(f"Exact-match F1:        {f1:.3f}\n")  

Exact-match Accuracy: 0.520
Exact-match Precision: 1.000
Exact-match Recall:    0.520
Exact-match F1:        0.685



In [13]:
preds, refs = [], []
for row in df.itertuples():
    img = Image.open(row.image_path).convert("RGB")
    prompt = f"Based on the image, answer the following question with a single word. Question: {row.question} Answer:"
    inputs = processor(images=img, text=prompt, return_tensors="pt").to(device)
    out = model.generate(**inputs, max_new_tokens=10)
    text = processor.batch_decode(out, skip_special_tokens=True)[0].strip()
    pred = text.split()[0].rstrip(".,;:!?") if text else ""
    preds.append(pred.lower())
    refs.append(row.answer.lower())

In [14]:
# ──────────────────────────────────────────────────────────────────────────────
# 6. Compute Metrics (and identify mismatches)
# ──────────────────────────────────────────────────────────────────────────────
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import evaluate
# from bert_score import BERTScorer # Keep if you add BERTScore later
# from rouge_score import rouge_scorer # Keep if you add ROUGE later
# from rapidfuzz.distance import Levenshtein # Keep if you add Levenshtein later
from sentence_transformers import SentenceTransformer, util # Keep if you add SBERT later
import numpy as np
import pandas as pd # Ensure pandas is imported if df is used



# normalize case
preds_l = [str(p).lower().strip() for p in preds] # Added str() and strip() for robustness
refs_l  = [str(r).lower().strip() for r in refs]   # Added str() and strip()

# 1) TOKEN-LEVEL EXACT MATCH → binary labels
y_pred_bin = [int(p == r) for p, r in zip(preds_l, refs_l)]
y_true_bin = [1]*len(refs_l)            # reference is always “correct” class

acc = accuracy_score(y_true_bin, y_pred_bin)
prec, rec, f1, _ = precision_recall_fscore_support(
    y_true_bin, y_pred_bin, average="binary", zero_division=0
)
print(f"Exact-match Accuracy: {acc:.3f}")
print(f"Exact-match Precision: {prec:.3f}") # Will be 1.0 if acc > 0, else 0.0
print(f"Exact-match Recall:    {rec:.3f}")    # Will be same as acc
print(f"Exact-match F1:        {f1:.3f}\n")  

Exact-match Accuracy: 0.505
Exact-match Precision: 1.000
Exact-match Recall:    0.505
Exact-match F1:        0.671

